# Higgs Boson ML Challenge

In [1]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from implementations import *
from helpers import *

%load_ext autoreload
%autoreload 2

## Load the dataset

In [ ]:
path_data_tr = 'data/train.csv'
path_data_te = 'data/test.csv'
path_submission = 'data/submission.csv'

y_tr, tx_tr, ids_tr = load_csv_data(path_data_tr, sub_sample=False)
y_te, tx_te, ids_te = load_csv_data(path_data_te)
y_tr = np.expand_dims(y_tr, axis=1)
y_pred=np.zeros(len(y_test))